Here is the MUCH more powerful network architecture. I will now test out this bad boy on a new game.

(OUTPUT LAYER DETAILS)
N1 = Up
N2 = Left
N3 = Down
N4 = Right




## MAKE SURE TOP NET IS SAVED AND DISTINGUISHED

In [1]:
import numpy as np

class layer:
    """
    Creates class that holds value, bias, and weights of all neurons in layer. 
    Has additional variables: first_layer (bool) and size (int)
    """
    def __init__(self, size, weights, is_first_layer = False):
        """
        Creates layer object. If first_layer is true will not have bias and weights
        """
        self.size = size
        self.value = np.zeros(size)
        self.first_layer = is_first_layer
        
        if not self.first_layer:
            self.bias = np.random.rand(size)*2-np.ones(size)
            self.weights = weights 
            
    def mutate_by_rate(self, weights, mutation_rate):
        """
        Changes weights and biases of self
        """
        self.bias += mutation_rate/10 * (np.random.rand(self.size)*2-np.ones(self.size))
        self.weights += mutation_rate/10 * weights
        
    def copy(self):
        """
        Returns copy of layer (same size, weights, bias, first_layer)
        """
        if self.first_layer:
            copycat = layer(self.size, 0, True)
            copycat.size = self.size
        else:
            copycat = layer(self.size, self.weights.copy())
            copycat.bias = self.bias.copy()
            
        return copycat
    
    def __eq__(self, other_layer):
        
        if self.size != other_layer.size:
            return False
        
        if self.first_layer == True or other_layer.first_layer == True:
            if self.first_layer == other_layer.first_layer:
                return True
            else:
                return False
            
        if not np.array_equal(self.bias, other_layer.bias):
            return False
        if not np.array_equal(self.weights, other_layer.weights):
            return False
        
        return True
    
class network:
    """
    Creates network to deal with layers. Has layers list and layers_tuple. 
    First object in layers is always layer with first_layer = True
    """
    def __init__(self, layers_tuple):
        """
        Create network object
        """
        self.layers = []
        self.layers_tuple = layers_tuple    
        
        first_time = True
        for size in layers_tuple:
            
            if first_time:
                self.layers.append(layer(size, 0, True))
                first_time = False
                continue
             
            self.layers.append(layer(size, np.random.rand(size, self.layers[-1].size)*2-np.ones((size, self.layers[-1].size))))
            
                
    def sigmoid(x):
        """
        Is function of class, returns sigmoid of value passed to it. Can take in array as well
        """
        sigm = 1 / (1 + np.exp(-x))
        return sigm 
        
    def calculate(self, input_array):
        """
        Returns output numpy array when given input numpy array
        """
        for index, layer in enumerate(self.layers):
            if layer.first_layer:
                layer.value = input_array
                continue
            
            layer.value = network.sigmoid(layer.bias + np.matmul(layer.weights, self.layers[index-1].value))
            
        return self.layers[-1].value.copy()

    def mutate_by_rate(self,mutation_rate=1.0):
        """
        PASS COPY OF SELF INTO METHOD
        at mutation_rate = 2 the difference is about 0.02 however after that it increases exponentially with 
        mutation_rate = 3 causing an difference of about 0.06
        """
        for index, layer in enumerate(self.layers):
            if layer.first_layer:
                continue 
            
            layer.mutate_by_rate(np.random.rand(layer.size, self.layers[index-1].size)*2-np.ones((layer.size, self.layers[index-1].size)), mutation_rate)
    
    def mutate_by_crossover(self, oppurtunity):
        """
        Not yet made
        """
        
        pass
                    
    def copy(self):
        """
        Returns copy of network
        """
        copycat = network(self.layers_tuple)
        for index, layer in enumerate(self.layers):
            copycat.layers[index] = self.layers[index].copy()
            
        return copycat
    
    def __str__(self):
        
        first_time = True
        for layer in self.layers:
            if layer.first_layer:
                continue
            
            print("New Layer!")
            print(layer.weights)
            print(layer.bias)
        return ""
    
    def __eq__(self, other_network):
        if not self.layers_tuple == other_network.layers_tuple:
            return False
        for layer, other_layer in zip(self.layers, other_network.layers):
            if not layer == other_layer:
                return False 
            
        return True
    
class population:
    """
    Is front of AI operations. Handles all work and uses network and layer classes for own purposes.
    Game class should only have to call methods off of this class. 
    Game class should store: pop (population); fitness (array 1 dim); input (array 2 dim)
    Game class should handle: output (array 2 dim)
    """
    def __init__(self, population_size, network_layers_tuple):
        """
        Create list of networks stored as residents (list); network_size (tuple) stores network layers_tuple.
        population_size stores number of residents and fitness stores fitness scores
        """
        self.network_size = network_layers_tuple
        self.fitness = np.zeros(population_size)
        self.population_size = population_size
        self.residents = [network(network_layers_tuple) for i in range(population_size)]
        
    def calculate(self, input_array):
        """
        Take in input 2d array. (format: array([first network inputs], [second network inputs], ...))
        Return output 2d array. (format: array([first network output], [second network output], ...))
        """
        output_array = np.zeros((self.population_size, self.network_size[-1]))
        
        for index, network in enumerate(self.residents):
            output_array[index] = network.calculate(input_array[index])
               
        return output_array
    
    def set_fitness(self, fitness_array):
        self.fitness = fitness_array
    
    def copy(self):
        """
        Returns copy of self
        """
        copycat = population(self.population_size, self.network_size)
        for index, resident in enumerate(self.residents):
            copycat.residents[index] = resident.copy()
            
        return copycat
    
    def mutate_by_best(self, mutation_rate = 1):
        """
        Input mutation_rate to specify how much top network should change
        Takes best and mutates it while adding in a population_size/6 randomly generated networks.
        """
        top_network = self.residents[self.fitness.argmax()]
        self.residents = []
        number_random = int(np.floor(np.array([self.population_size])/6))
        
        self.residents.append(top_network.copy())
        for i in range(number_random):
            self.residents.append(network(self.network_size))
                            
        for i in range(self.population_size - number_random-1):
            top_net_copy = top_network.copy()
            top_net_copy.mutate_by_rate(mutation_rate)
            self.residents.append(top_net_copy)
        
        self.fitness = np.zeros(self.population_size)
    
    def mutate_by_fitness(self, mutation_rate=1):
        """
        Input mutation_rate to specify how much selected network should change
        Adds population_size/6 random networks and then mutates network that is selected from self.residents
        However networks with higher fitness have greater chance of getting selected 
        """
        number_random = int(np.floor(np.array([self.population_size])/6))
        new_residents = []
        new_residents.append(self.residents[self.fitness.argmax()])
        
        for i in range(number_random):
            new_residents.append(network(self.network_size))
        
        for i in range(self.population_size - number_random - 1):
            
            total = np.random.rand(1)*np.sum(self.fitness)
            
            selected_index = "False"
            
            for i in range(len(self.fitness)):
                if np.sum(self.fitness[:i+1]) > total:
                    selected_index = i 
                    break
            
            selected_net = self.residents[selected_index].copy()
            selected_net.mutate_by_rate(mutation_rate)
            new_residents.append(selected_net)
        
        self.residents = new_residents
        
        
    def mutate_by_crossover(self):
        pass
                

In [2]:
import pygame
from pygame.locals import *

class wall:
    
    def __init__(self, center_xcoor, center_ycoor, width, height):
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(center_xcoor-width/2,center_ycoor-height/2,width,height)
        self.pos = pygame.math.Vector2(center_xcoor, center_ycoor)
        
    
    def show(self, gameDisplay):
        pygame.draw.rect(gameDisplay, (255,255,255), self.hitbox)
        
class bot:
    
    def __init__(self, speed=2, color = (255,0,0), pos =pygame.math.Vector2(20,10), width =20, height =20):
        self.lock_frames = 0
        self.is_alive = True
        self.speed = speed
        self.color = color
        self.pos = pos
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(pos[0]-width/2, pos[1]-height/2, width, height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
    def show(self, gameDisplay, color = (0,0,0)):
        rect = (int(self.hitbox.topleft[0]), int(self.hitbox.topleft[1]), self.width, self.height)
        if color != (0,0,0):
            pygame.draw.rect(gameDisplay, color, rect)
        else:
            pygame.draw.rect(gameDisplay, self.color, rect)
    
    def stun(self, num_frames):
        self.lock_frames = num_frames
        
    def get_movement(self, player):
        
        if self.lock_frames != 0:
            self.lock_frames -= 1
        else:
            difference = player.pos - self.pos

            if difference[0] < 0:
                self.movement_vector += pygame.math.Vector2(-1,0)
            elif difference[0] == 0:
                self.movement_vector += pygame.math.Vector2(0,0)
            elif difference[0] > 0:
                self.movement_vector += pygame.math.Vector2(1,0)

            if difference[1] < 0:
                self.movement_vector += pygame.math.Vector2(0,-1)
            elif difference[1] == 0:
                self.movement_vector += pygame.math.Vector2(0,0)
            elif difference[1] > 0:
                self.movement_vector += pygame.math.Vector2(0,1)

            if self.movement_vector != pygame.math.Vector2(0,0):
                self.movement_vector.normalize_ip()

    def update(self):
        self.pos += self.movement_vector*self.speed
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
    
    def reset(self):
        self.is_alive = True
        self.pos = pygame.math.Vector2(20,10)
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
class player:
    
    def __init__(self, speed =3, color = (0,0,255), pos =pygame.math.Vector2(250,500), width =20, height =20):
        self.is_alive = True
        self.speed = speed
        self.color = color
        self.pos = pos
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(pos[0]-width/2, pos[1]-height/2, width, height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
    def show(self, gameDisplay, color = (0,0,0)):
        rect = (int(self.hitbox.topleft[0]), int(self.hitbox.topleft[1]), self.width, self.height)
        if color != (0,0,0):
            pygame.draw.rect(gameDisplay, color, rect)
        else:
            pygame.draw.rect(gameDisplay, self.color, rect)
        
    def get_movement(self, output_array):
        
        if output_array.argmax() == 0:
            self.movement_vector += pygame.math.Vector2(0,-1)
        if output_array.argmax() == 1:
            self.movement_vector += pygame.math.Vector2(-1,0)
        if output_array.argmax() == 2:
            self.movement_vector += pygame.math.Vector2(0,1)
        if output_array.argmax() == 3:
            self.movement_vector += pygame.math.Vector2(1,0)
            
        if self.movement_vector != pygame.math.Vector2(0,0):
            self.movement_vector.normalize_ip()
            
    def update(self):
        self.pos += self.movement_vector*self.speed
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)
    
    def reset(self):
        self.is_alive = True
        self.pos = pygame.math.Vector2(250,500)
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def collision(square1, square2):
    return square1.hitbox.colliderect(square2.hitbox)  

def display_message(gameDisplay, text):
    text_font = pygame.font.Font("freesansbold.ttf", 30)
    textDisplaySurface = text_font.render(text, False, (0,255,255))
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (400, 300)
    gameDisplay.blit(textDisplaySurface, text_rect)
    pygame.display.update()
    
def boundry_collision(square):
    return (int(square.pos[0]) < 0+square.width/2 or int(square.pos[0]) > 800-square.width/2 or int(square.pos[1]) < 0+square.height/2 or int(square.pos[1]) > 600-square.height/2)
    
def display_strats(strat, center_x, center_y, text_size = 12):
    textfont = pygame.font.Font('freesansbold.ttf', text_size)
    textSurface = textfont.render(strat, True, (0,255,0))
    textRect = textSurface.get_rect()
    textRect.center = (center_x, center_y)
    gameDisplay.blit(textSurface, textRect)

#AI Variables
population_number = 200
network_tuple = (8,16,16,4) #Inputs: x,y dif to wall, selfxpos, selfypos, x,y dif to bot, botxpos, botypos
pop = population(population_number, network_tuple)
fitness = np.zeros(population_number)
num_nets = population_number
generation_number = 1
highest_fitness_so_far = 0
improvement = 0
highest_fitness_overall = 0
best_net = network(network_tuple)


#Pygame Variables
pygame.init()
pygame.display.set_caption("NN ATTEMPT BOIS")
gameDisplay = pygame.display.set_mode((800,600))
clock = pygame.time.Clock()

#Level Variables
all_players = [player(2, ((1*i),(1*i), 255)) for i in range(population_number)]
all_bots = [bot(2.25, (255,(1*i),(1*i))) for i in range(population_number)]
wall1 = wall(400,300,50,50)

#Starting Variables
start_ticks = pygame.time.get_ticks()
run = True
cont = False
while run:
    
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
        elif event.type == KEYDOWN:
            if event.key == pygame.K_SPACE:
                run = False
            if event.key == pygame.K_p:
                if cont:
                    cont = False
                else:
                    cont = True
    
    if cont:
        display_strats("Paused!", 400, 300, 50)
        pygame.display.update()
        continue
        
    
    #Start process for every network
    for iden, net in enumerate(pop.residents):
        if all_bots[iden].is_alive:
            #Get the movements from all and update positions for all
            all_bots[iden].get_movement(all_players[iden])
            all_bots[iden].update()  
            pwdif = all_players[iden].pos - wall1.pos
            pbdif = all_players[iden].pos - all_bots[iden].pos
            output_array = net.calculate(np.array([pwdif[0], pwdif[1], all_players[iden].pos[0], all_players[iden].pos[1], pbdif[0], pbdif[1], all_bots[iden].pos[0], all_bots[iden].pos[1]]))
            all_players[iden].get_movement(output_array)
            all_players[iden].update()

            #Boundry logic
            #First check collision with boundries
            if boundry_collision(all_bots[iden]):
                all_bots[iden].get_movement(all_players[iden])

            if boundry_collision(all_players[iden]):
                all_bots[iden].is_alive = False
                all_players[iden].is_alive = False
                fitness[iden] = (pygame.time.get_ticks()-start_ticks)/1000
                num_nets -= 1
                
            #Then check collision between all three objects
            if collision(all_bots[iden], wall1):
                difference = all_bots[iden].pos - wall1.pos
                total_width = all_bots[iden].width + wall1.width 
                total_height = all_bots[iden].height + wall1.height
                if difference[0] < 0:
                    movex = -abs(difference[0] - total_width)
                else:
                    movex = abs(difference[0] - total_width)
                if difference[1] < 0:
                    movey = -abs(difference[1] - total_height)
                else:
                    movey = abs(difference[1] - total_height)

                all_bots[iden].movement_vector = pygame.math.Vector2(movex, movey)
                all_bots[iden].movement_vector.normalize_ip()
                all_bots[iden].stun(30)

            if collision(all_bots[iden], all_players[iden]):
                all_bots[iden].is_alive = False
                all_players[iden].is_alive = False
                fitness[iden] = (pygame.time.get_ticks()-start_ticks)/1000
                num_nets -= 1

            if collision(all_players[iden], wall1):
                all_bots[iden].is_alive = False
                all_players[iden].is_alive = False
                fitness[iden] = (pygame.time.get_ticks()-start_ticks)/1000
                num_nets -= 1     


    #Visuals
    gameDisplay.fill((0,0,0))
    for iden, net in enumerate(pop.residents):
        if best_net == net:
            continue
        if all_bots[iden].is_alive:
            all_bots[iden].show(gameDisplay)
            all_players[iden].show(gameDisplay)
    
    for iden, net in enumerate(pop.residents):
        if best_net == net:
            all_bots[iden].show(gameDisplay, (255,255,0))
            all_players[iden].show(gameDisplay, (0,255,255))
            
    wall1.show(gameDisplay)
    display_strats("The time so far is: " + str(round((pygame.time.get_ticks()-start_ticks)/1000, 2)), 700, 25)
    display_strats("Num AI Left: " + str(num_nets), 700, 50)
    display_strats("Highest Fitness: " + str(round(highest_fitness_overall,2)), 700, 75)
    display_strats("Generation Number: " + str(generation_number), 700, 100)
    display_strats("Improvement : " + str(round(improvement, 2)), 700, 125) 
        
    #Check if all dead, if True, start process for next generation
    if num_nets == 0:
        
        improvement = fitness.max() - highest_fitness_so_far       
        if highest_fitness_so_far > highest_fitness_overall:
            highest_fitness_overall = highest_fitness_so_far
        highest_fitness_so_far = fitness.max()
        
        pop.set_fitness(fitness)
        best_net = pop.residents[pop.fitness.argmax()].copy()
        pop.mutate_by_fitness(3)
        for iden in range(population_number):
            all_players[iden].reset()
            all_bots[iden].reset()
            
        num_nets = population_number
        generation_number += 1
        fitness = np.zeros(population_number)
        
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "New Gen Start!")
        pygame.time.wait(2000)
    
        start_ticks = pygame.time.get_ticks()
    
    pygame.display.update()
    clock.tick(60)
            
pygame.quit()

C:\Users\megat\AppData\Local\Temp\ipykernel_27872\870702611.py:85: RuntimeWarning: overflow encountered in exp
  sigm = 1 / (1 + np.exp(-x))
